# Step 8: Machine Learning/Deep Learning Prototype

## Learning Objective

- Find a machine learning or deep learning approach that works for the problem to be solved.
- Implement a prototype of the approach in a Jupyter notebook. 
- Demonstrate the viability of the approach.

### Prerequisites
- Python 3.9 
- Tensorflow - installed via Anaconda (https://www.anaconda.com/products/individual)
- Download yolov4.weights file: https://drive.google.com/open?id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT
and place in "../yolo/weights" folder
- opencv

Setup of folders and file paths used throughout this notebook. 
Note: this notebook calls a files/folders that are one folder above it. For it work, the entire git repo must be checked out as it uses submodules

If needed, set "get_requirements = True" to install necessary requirements used:

In [1]:
get_requirements = False
if (get_requirements == True):
    !pip install -r ../requirements.txt

Importing necessary libraries:

In [2]:
import os
import shutil
import pandas as pd

Variables Setup: 

In [3]:
# setup
dataset_dir_path = '../../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_050000_07_001014_001126'
video_name_orig = video_name + video_ext
video_dest_path = './../processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt'
yolo_video_name = 'yolo'

# annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


src_video = os.path.join(video_src_path, video_name_orig)
gt_video_name = yml_video_name + '_' # + '.mp4'#video_ext
yolo_video_name = yolo_video_name + '_'# + '.mp4'#video_ext

Copy folders that will be used later:

In [4]:

def CopyFolders(src, dest):
    ''' This is a helper function to copy entire folders'''
    if os.path.exists(dest): 
        shutil.rmtree(dest) 
    shutil.copytree(src,dest)
    print(f'Copied folder {src} --> {dest}')

# copy folders that will be used by the algorithm
CopyFolders('../tensorflow_yolov4_tflite/data/', '../data') 
CopyFolders('../tensorflow_yolov4_tflite/core/', '../core') 
CopyFolders('../yolov4-deepsort/model_data/','../model_data') 


Copied folder ../tensorflow_yolov4_tflite/data/ --> ../data
Copied folder ../tensorflow_yolov4_tflite/core/ --> ../core
Copied folder ../yolov4-deepsort/model_data/ --> ../model_data


Using https://github.com/zyerusha/tensorflow-yolov4-tflite to convert YOLO weights to tensorflow:


In [5]:
# If not done already, Convert weights to tensorflow model
if not os.path.exists("../checkpoints/yolov4"):
    current_dir  = os.getcwd()
    os.chdir("../tensorflow_yolov4_tflite/")
    !python save_model.py --model yolov4 --weights ../yolo/weights/yolov4.weights --output ../checkpoints/yolov4
    os.chdir(current_dir)


The following code creates/loads a 

In [10]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)

if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])

type_df

The following code loads a dataframe with annotation data from either a yaml or from a csv file. The csv is created after the first yaml reading on the data. This provides  faster loading of data on a rerun of this notebook.

In [9]:
# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head(10)

Loading annotations...


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category
0,0.0,0.0,0.000000,485,743,653,914,Vehicle
1,0.0,1.0,0.033333,489,748,657,919,Vehicle
2,0.0,2.0,0.066667,488,747,656,918,Vehicle
3,0.0,3.0,0.100000,488,747,656,918,Vehicle
4,0.0,4.0,0.133333,488,747,656,918,Vehicle
5,0.0,5.0,0.166667,488,746,656,917,Vehicle
6,0.0,6.0,0.200000,488,746,656,917,Vehicle
7,0.0,7.0,0.233333,488,746,656,917,Vehicle
8,0.0,8.0,0.266667,488,745,656,916,Vehicle
9,0.0,9.0,0.300000,488,745,656,916,Vehicle
